In [38]:
import json
import re
import pandas as pd

def parse_json(file_path):
    # Загружаем JSON файл
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Будем использовать список для хранения информации
    extracted_data = []

    for item in data:
        # Проверяем заголовок задания
        if item.get("title") == "Расставьте знаки препинания.":
            # Извлекаем описание и определяем нужный знак препинания
            description = item.get("description", "")
            punctuation_sign = extract_punctuation_sign(description)
            
            # Извлекаем само задание и ответ ученика
            task_text = item.get("task", "")
            student_answer = item.get("answer", "")
            answer = item.get("input_answer", "")
            
            # Сохраняем данные в виде словаря
            extracted_data.append({
                "description": description,
                "punctuation_sign": punctuation_sign,
                "task_text": task_text,
                "student_answer": student_answer,
                "answer": answer
            })

    # Конвертируем список в DataFrame для удобства анализа
    df = pd.DataFrame(extracted_data)
    return df

def extract_punctuation_sign(description):
    """Функция для извлечения знака препинания из описания задания."""
    if "запятые" in description.lower():
        return ","
    elif "тире" in description.lower():
        return "—"
    elif "двоеточие" in description.lower():
        return ":"
    elif "точка с запятой" in description.lower():
        return ";"
    return None

# Пример вызова функции
file_path = 'dataset.json'
df = parse_json(file_path)


import re
from transformers import pipeline, AutoTokenizer
import logging




# Отключаем все логирование
logging.disable(logging.CRITICAL)


# Загрузка модели для расстановки пунктуации
pt = "RUPunct/RUPunct_big"

tk = AutoTokenizer.from_pretrained(pt, strip_accents=False, add_prefix_space=True)
classifier = pipeline("ner", model=pt, tokenizer=tk, aggregation_strategy="first")


import re

def process_token(token, label, prev_char, is_start_of_sentence):
    """Обработка токенов в зависимости от их метки без изменения регистра, с капитализацией в начале предложения."""
    new_token = token
    substitution = None

    if label == "LOWER_O":
        return token, None
    if label in ["LOWER_COMMA", "UPPER_COMMA", "UPPER_TOTAL_COMMA"]:
        if prev_char != ',':
            new_token += ","
            substitution = ","
    elif label in ["LOWER_QUESTION", "UPPER_QUESTION", "UPPER_TOTAL_QUESTION"]:
        if prev_char != '?':
            new_token += "?"
            substitution = "?"
    elif label == "LOWER_TIRE":
        new_token = " " + token + " —"
        substitution = " —"
    elif label == "UPPER_TIRE":
        new_token = " " + token.capitalize() + " —"
        substitution = " —"
    elif label == "LOWER_DVOETOCHIE":
        new_token += ":"
        substitution = ":"
    elif label == "LOWER_VOSKL":
        new_token += "!"
        substitution = "!"
    elif label == "LOWER_PERIODCOMMA":
        new_token += ";"
        substitution = ";"
    elif label == "LOWER_DEFIS":
        new_token += "-"
        substitution = "-"
    elif label == "LOWER_MNOGOTOCHIE":
        new_token += "..."
        substitution = "..."
    elif label == "LOWER_QUESTIONVOSKL":
        new_token += "?!"
        substitution = "?!"
    elif label == "UPPER_O":
        new_token = token.capitalize()
    elif label == "UPPER_TOTAL_O":
        new_token = token.upper()

    # Если это начало предложения, делаем первую букву заглавной
    if is_start_of_sentence and new_token:
        new_token = new_token.capitalize()

    return new_token, substitution

def punctuate_and_get_symbols(sentence):
    """Функция для расстановки пунктуации и генерации списка вставленных знаков для одного предложения."""
    print("Исходное предложение:", sentence)
    modified_sentence = re.sub(r'\(\d+\)', '', sentence).strip()
    print("После удаления меток:", modified_sentence)
    modified_sentence = re.sub(r'[^\w\s]', '', modified_sentence)

    preds = classifier(modified_sentence)

    output = ""
    prev_char = ""
    symbols = {}
    symbol_counter = 1  # Счетчик для нумерации вставленных знаков

    for index, item in enumerate(preds):
        is_start_of_sentence = (index == 0 or prev_char in ".!?")  # Определяем, является ли токен началом предложения
        token, substitution = process_token(item['word'].strip(), item['entity_group'], prev_char, is_start_of_sentence)

        if substitution:
            symbols[symbol_counter] = substitution
            symbol_counter += 1  # Увеличиваем счетчик только при добавлении знака

        if token and token[0] not in ",!?;:":  # Не добавляем лишние пробелы
            output += " " + token
        else:
            output += token

        prev_char = token[-1] if token else prev_char

    output = output.strip()
    print("Финальное предложение:", output)
    return output, symbols



def process_text(input_text):
    """Функция для обработки всего текста, разбивая его на предложения."""
    sentences = re.split(r'(?<=[.!?]) +', input_text)
    processed_sentences = []
    all_symbols = {}

    for i, sentence in enumerate(sentences):
        processed_sentence, symbols = punctuate_and_get_symbols(sentence)

        # Добавляем точку в конце, если она отсутствует
        if processed_sentence and processed_sentence[-1] not in ".!?":
            processed_sentence += "."
        
        processed_sentences.append(processed_sentence)

        for num, symbol in symbols.items():
            all_symbols[len(all_symbols) + 1] = symbol  # Нумерация знаков по порядку

    final_output = ' '.join(processed_sentences)
    return final_output, all_symbols

def extract_punctuation_sign(task_description):
    """Функция для извлечения знака препинания из задания."""
    if "запятые" in task_description.lower():
        return ","
    elif "тире" in task_description.lower():
        return "—"
    elif "двоеточие" in task_description.lower():
        return ":"
    elif "точка с запятой" in task_description.lower():
        return ";"
    return None

def find_punctuation(input_text, task_description):
    """Функция для нахождения позиций нужного знака."""
    required_sign = extract_punctuation_sign(task_description)
    if required_sign is None:
        return []

    processed_text, symbols = process_text(input_text)
    
    # Получаем все метки из текста, даже если они пусты
    markers = sorted(int(num) for num in re.findall(r'\((\d+)\)', input_text))
    
    # Заполняем позиции, включая "N" для пустых меток
    all_positions = {marker: symbols.get(marker, 'N') for marker in markers}

    return processed_text, all_positions

def compare_punctuation(input_text, punctuation_indices):
    """Сравнивает, какие метки были заменены на какие знаки."""
    markers = re.findall(r'\((\d+)\)', input_text)

    changes = {}
    for marker in markers:
        marker_num = int(marker)
        original_marker = f"({marker_num})"
        added_symbol = punctuation_indices.get(marker_num, "N")

        # Сопоставление замены
        changes[original_marker] = added_symbol

    return changes

def extract_labels(text):
    """Извлекает метки в формате (число) из текста."""
    labels = re.findall(r'\(\d+\)', text)
    return labels

def compare_labels(original_text, modified_text):
    """Сравнивает метки и определяет, что с ними произошло в модифицированном тексте."""
    changes = {}
    print(f"Оригинальный текст: {original_text}")
    print(f"Изменённый текст: {modified_text}")

    # Разделяем оригинальный текст на слова
    original_words = original_text.split()
    print(f"Слова в оригинальном тексте: {original_words}")

    # Проходим по словам и ищем метки
    for i in range(len(original_words)):
        word = original_words[i]
        print(f"\nОбрабатываем слово: {word} (Индекс {i})")
        
        # Если это метка
        if '(' in word and ')' in word:
            label = word  # Сохраняем метку
            print(f"Найдена метка: {label}")

            # Берем слово после метки как правую границу
            right_word = original_words[i + 1] if i < len(original_words) - 1 else ''
            print(f"Слово справа от метки: {right_word}")

            # Ищем правую границу в изменённом тексте
            right_index = modified_text.find(right_word)
            print(f"Индекс правой границы в изменённом тексте: {right_index}")

            if right_index == -1:
                changes[label] = 'Nan'  # Если правое слово не найдено
                print(f"{label}: Правое слово не найдено, устанавливаем значение 'Nan'")
                continue

            # Ищем левую границу
            left_word = original_words[i - 1] if i > 0 else ''
            print(f"Слово слева от метки: {left_word}")

            left_index = modified_text.rfind(left_word, 0, right_index)
            print(f"Индекс левой границы в изменённом тексте: {left_index}")

            if left_index == -1:
                changes[label] = 'Nan'  # Если левое слово не найдено
                print(f"{label}: Левое слово не найдено, устанавливаем значение 'Nan'")
                continue
            
            # Проверяем расстояние между границами
            while True:
                # Ищем пробелы и специальные символы между границами
                distance = right_index - (left_index + len(left_word))  # Рассчитываем расстояние
                print(f"Расстояние между границами: {distance}")

                if distance <= 3:  # Если расстояние корректное
                    # Извлекаем текст между словами
                    text_between = modified_text[left_index + len(left_word):right_index].strip()
                    print(f"Текст между границами: '{text_between}'")

                    # Проверяем наличие специальных знаков
                    if any(char in text_between for char in " ,:;-—"):
                        changes[label] = text_between
                        print(f"{label}: Найден текст со специальными символами: '{text_between}'")
                    else:
                        changes[label] = 'Nan'  # Если специальные знаки не найдены
                        print(f"{label}: Специальные символы не найдены, устанавливаем значение 'Nan'")
                    break  # Выходим из цикла после успешного извлечения текста

                # Если расстояние слишком большое, ищем новую левую границу
                left_index = modified_text.rfind(left_word, 0, left_index)  # Ищем левую границу заново
                print(f"Новый индекс левой границы: {left_index}")
                
                if left_index == -1 or left_index >= right_index:
                    changes[label] = 'Nan'  # Если не нашли соответствия
                    print(f"{label}: Левое слово не найдено или находится в некорректной позиции, устанавливаем значение 'Nan'")
                    break  # Выходим из цикла, если левое слово не найдено

                left_word = modified_text[left_index:left_index + len(original_words[i - 1])].strip()
                print(f"Новое слово для левой границы: {left_word}")

    print(f"\nИзменения: {changes}")
    return changes


def process_task_text(row):
    input_text = row['task_text']
    task_description = row['description']

    # Применяем ваш алгоритм
    processed_text, punctuation_indices = find_punctuation(input_text, task_description)

    label_changes = compare_labels(input_text, processed_text)
    
    # Возвращаем как кортеж или словарь, если нужно больше данных
    return processed_text, label_changes  # Или любой другой формат

# Обновляем DataFrame для хранения нескольких выходных данных
df[['model_answer', 'punctuation_indices']] = df.apply(process_task_text, axis=1, result_type='expand')



Исходное предложение: Суздальский музей деревянного зодчества (1) настоящий городок (2) построенный без единого гвоздя.
После удаления меток: Суздальский музей деревянного зодчества  настоящий городок  построенный без единого гвоздя.
Финальное предложение: Суздальский музей деревянного зодчества Настоящий городок, построенный без единого гвоздя
Исходное предложение: Из дерева здесь всё (3) ложки и лавки в избе, и даже крытая лемехом церковная маковка.
После удаления меток: Из дерева здесь всё  ложки и лавки в избе, и даже крытая лемехом церковная маковка.
Финальное предложение: Из дерева здесь всё: ложки, и лавки в избе, и даже крытая лемехом церковная маковка
Исходное предложение: Мельницы (4) церковь (5) дома́ (6) амбары и бани (7) всё привезено сюда из разных сёл Владимирской области и поставлено на территории несохранившегося Дмитриевского монастыря.
После удаления меток: Мельницы  церковь  дома́  амбары и бани  всё привезено сюда из разных сёл Владимирской области и поставлено на 

In [39]:
df

,description,punctuation_sign,task_text,student_answer,answer,model_answer,punctuation_indices
0,"Укажите цифры, на месте которых в предложении ...",—,Суздальский музей деревянного зодчества (1) на...,1235678,17,Суздальский музей деревянного зодчества Настоя...,"{'(1)': 'Nan', '(2)': ',', '(3)': ':', '(4)': ..."
1,"Укажите цифры, на месте которых в предложении ...",",",Кремль (1) самая древняя часть столицы России ...,234568,234568,"кремль — самая древняя часть столицы россии, р...","{'(1)': 'Nan', '(2)': 'Nan', '(3)': 'Nan', '(4..."
2,"Укажите цифры, на месте которых в предложении ...",",",Дерево (1) материал недолговечный (2) а время ...,2467,2467,"дерево — материал недолговечный, а время и пож...","{'(1)': 'Nan', '(2)': 'Nan', '(3)': 'Nan', '(4..."
3,"Укажите цифры, на месте которых в предложении ...",—,Колокольня Ивана Великого (1) это церковь из б...,17,13,Колокольня ивана Великого — это церковь из бе...,"{'(1)': '—', '(2)': ',', '(3)': 'Nan', '(4)': ..."
4,"Укажите цифры, на месте которых в предложении ...",",",Заядлые путешественники (1) ищущие (2) что пос...,129,12359,"Заядлые путешественники ищущие, что посмотреть...","{'(1)': 'Nan', '(2)': ',', '(3)': ',', '(4)': ..."
5,"Укажите цифры, на месте которых в предложении ...",—,Художественные изделия из берёсты (1) оригинал...,56,17,Художественные изделия из берёсты — оригиналь...,"{'(1)': '—', '(2)': ',', '(3)': 'Nan', '(4)': ..."
6,"Укажите цифры, на месте которых в предложении ...",",",Госудáрственная Третьякóвская галерéя (1) моск...,67,23489,Госудáрственная третьякóвская галерéя — Моско...,"{'(1)': 'Nan', '(2)': ',', '(3)': 'Nan', '(4)'..."
7,"Укажите цифры, на месте которых в предложении ...",—,Национальный театр драмы имени известного алта...,12345,16,Национальный театр драмы имени известного алта...,"{'(1)': '—', '(2)': ',', '(3)': 'Nan', '(4)': ..."
8,"Укажите цифры, на месте которых в предложении ...",",",Дом-музей Петра I в Вологде (1) именуемый «Пет...,123,123568,"Доммузей петра I в вологде, именуемый Петровск...","{'(1)': 'Nan', '(2)': 'Nan', '(3)': 'Nan', '(4..."
9,"Укажите цифры, на месте которых в предложении ...",—,Домик Петра I (1) единственная постройка (2) с...,17,17,"Домик петра i единственная постройка, сохранив...","{'(1)': 'Nan', '(2)': ',', '(3)': 'Nan', '(4)'..."


In [42]:
def filter_punctuation_indices(row):
    sign = row['punctuation_sign']
    indices = row['punctuation_indices']

    # Фильтруем индексы по заданному знаку пунктуации
    filtered_indices = [key[1] for key, value in indices.items() if value == sign]

    # Объединяем индексы в строку
    return ''.join(filtered_indices)

# Применяем фильтрацию к каждому ряду датафрейма
df['filtered_punctuation_indices'] = df.apply(filter_punctuation_indices, axis=1)

# Сравнение значений и создание нового столбца 'mark'
df['mark'] = df.apply(lambda row: f"{sum(1 for i in row['filtered_punctuation_indices'] if i in row['student_answer'])}/{len(row['student_answer'])}" if len(row['student_answer']) > 0 else "0/0", axis=1)

df['procent'] = df.apply(
    lambda row: (sum(1 for i in row['filtered_punctuation_indices'] if i in row['answer']) / len(row['answer']) * 100) if len(row['answer']) > 0 else 0, 
    axis=1
)

# Округляем до ближайшего целого числа и преобразуем в целочисленный тип
df['procent'] = df['procent'].round(0).astype(int)

# Формирование JSON выходных данных
assignments = []
for index, row in df.iterrows():
    assignment = {
        "id": index + 1,  # Индексация с 1
        "mark": row['mark'],
        "feedback": row['procent']
    }
    assignments.append(assignment)

output_json = {
    "assignments": assignments
}

# Сохранение в файл JSON
output_file = 'output.json'
with open(output_file, 'w') as json_file:
    json.dump(output_json, json_file, indent=3)

print(f'Данные успешно сохранены в файл {output_file}.')

import numpy as np
df['filtered_punctuation_indices'].replace('', np.nan, inplace=True)
df['filtered_punctuation_indices'] = df['filtered_punctuation_indices'].fillna(0)

df


Данные успешно сохранены в файл output.json.


,description,punctuation_sign,task_text,student_answer,answer,model_answer,punctuation_indices,filtered_punctuation_indices,mark,procent
0,"Укажите цифры, на месте которых в предложении ...",—,Суздальский музей деревянного зодчества (1) на...,1235678,17,Суздальский музей деревянного зодчества Настоя...,"{'(1)': 'Nan', '(2)': ',', '(3)': ':', '(4)': ...",0,0/7,0
1,"Укажите цифры, на месте которых в предложении ...",",",Кремль (1) самая древняя часть столицы России ...,234568,234568,"кремль — самая древняя часть столицы россии, р...","{'(1)': 'Nan', '(2)': 'Nan', '(3)': 'Nan', '(4...",46,2/6,33
2,"Укажите цифры, на месте которых в предложении ...",",",Дерево (1) материал недолговечный (2) а время ...,2467,2467,"дерево — материал недолговечный, а время и пож...","{'(1)': 'Nan', '(2)': 'Nan', '(3)': 'Nan', '(4...",0,0/4,0
3,"Укажите цифры, на месте которых в предложении ...",—,Колокольня Ивана Великого (1) это церковь из б...,17,13,Колокольня ивана Великого — это церковь из бе...,"{'(1)': '—', '(2)': ',', '(3)': 'Nan', '(4)': ...",1,1/2,50
4,"Укажите цифры, на месте которых в предложении ...",",",Заядлые путешественники (1) ищущие (2) что пос...,129,12359,"Заядлые путешественники ищущие, что посмотреть...","{'(1)': 'Nan', '(2)': ',', '(3)': ',', '(4)': ...",2359,2/3,80
5,"Укажите цифры, на месте которых в предложении ...",—,Художественные изделия из берёсты (1) оригинал...,56,17,Художественные изделия из берёсты — оригиналь...,"{'(1)': '—', '(2)': ',', '(3)': 'Nan', '(4)': ...",17,0/2,100
6,"Укажите цифры, на месте которых в предложении ...",",",Госудáрственная Третьякóвская галерéя (1) моск...,67,23489,Госудáрственная третьякóвская галерéя — Моско...,"{'(1)': 'Nan', '(2)': ',', '(3)': 'Nan', '(4)'...",289,0/2,60
7,"Укажите цифры, на месте которых в предложении ...",—,Национальный театр драмы имени известного алта...,12345,16,Национальный театр драмы имени известного алта...,"{'(1)': '—', '(2)': ',', '(3)': 'Nan', '(4)': ...",16,1/5,100
8,"Укажите цифры, на месте которых в предложении ...",",",Дом-музей Петра I в Вологде (1) именуемый «Пет...,123,123568,"Доммузей петра I в вологде, именуемый Петровск...","{'(1)': 'Nan', '(2)': 'Nan', '(3)': 'Nan', '(4...",68,0/3,33
9,"Укажите цифры, на месте которых в предложении ...",—,Домик Петра I (1) единственная постройка (2) с...,17,17,"Домик петра i единственная постройка, сохранив...","{'(1)': 'Nan', '(2)': ',', '(3)': 'Nan', '(4)'...",0,0/2,0


In [43]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Функция для расчета метрик для каждой группы
def calculate_metrics_for_group(group):
    # Убираем строки с пропусками
    group = group.dropna(subset=['answer', 'filtered_punctuation_indices'])
    
    y_true = list(map(int, group['answer']))
    y_pred = list(map(int, group['filtered_punctuation_indices']))


    # Метрики точности, полноты и F1
    precision = precision_score(y_true, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=1)
    
    return pd.Series({
        'Precision': precision * 100,
        'Recall': recall * 100,
    })

# Группировка по значению punctuation_sign и расчет метрик
metrics_by_group = df.groupby('punctuation_sign').apply(calculate_metrics_for_group)

# Вывод метрик для каждой группы
print(metrics_by_group)


                  Precision     Recall
punctuation_sign                      
,                 53.191489  50.000000
:                 50.000000  50.000000
—                 63.636364  65.454545
